In [46]:
# importing all packages
import great_expectations as gx
import psycopg2
import pandas as pd
import numpy as np
import logging
import csv
import json

# Import the 'config' funtion from the config.py file
from config import config

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)-8s %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger(__name__)

logger.handlers = []
logger.setLevel(logging.INFO)


params = config()
conn = psycopg2.connect(**params)
cur = conn.cursor()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [47]:
donations_mv = create_pandas_table("SELECT * FROM donations_mv")

2023-08-14 09:12:16 WARNING  /var/folders/96/_7q9mlsj0vl4n2jyl57mk42h0000gn/T/ipykernel_95283/1660043112.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table = pd.read_sql_query(sql_query, database)



In [ ]:
# read in the data tables
donations_mv = create_pandas_table("SELECT * FROM donations_mv")

# convert amount to numeric from string, removing dollar sign and comma
donations_mv['amount'] = donations_mv['amount'].str.replace('$', '', regex=True)
donations_mv['amount'] = donations_mv['amount'].str.replace(',', '', regex=True)
donations_mv['amount'] = pd.to_numeric(donations_mv['amount'])

# create "expect total" variable which we will use later on
donations_mv = donations_mv.assign(expect_total = donations_mv['amount_monetary'] + donations_mv['amount_non_monetary'])

# convert total and amount to numeric
donations_mv['expect_total'] = pd.to_numeric(donations_mv['expect_total'])

# add column with difference between amount and expected total
donations_mv = donations_mv.assign(diff = donations_mv['amount']-donations_mv['expect_total'])

# add absolute diff column
donations_mv['diff_abs'] = donations_mv['diff'].abs()

In [ ]:
# Close the cursor and connection to so the server can allocate
# bandwidth to other requests
cur.close()
conn.close()

https://github.com/great-expectations/great_expectations/blob/develop/docs_rtd/guides/tutorials/explore_expectations_in_a_notebook.rst

In [ ]:
# to run expectations, data need to be defined like this
donations_mv = gx.from_pandas(donations_mv)

In [ ]:
##### EXPECTATION 1 #####
# expectations for nothing to be null in any column of donations details
donations_mv.expect_column_values_to_not_be_null(column='amount')

In [ ]:
##### EXPECTATION 1 #####
# expectations for nothing to be null in any column of donations details
donations_mv.expect_column_values_to_not_be_null(column='donor_full_name')

In [ ]:
##### EXPECTATION 1 #####
# expectations for nothing to be null in any column of donations details
donations_mv.expect_column_values_to_not_be_null(column='political_party')

In [ ]:
##### EXPECTATION 1 #####
# expectations for nothing to be null in any column of donations details
donations_mv.expect_column_values_to_not_be_null(column='region')

In [ ]:
##### EXPECTATION 1 #####
# expectations for nothing to be null in any column of donations details
donations_mv.expect_column_values_to_not_be_null(column='donation_date',
                                                row_condition = 'region in ["Federal", "Ontario", "British Columbia"]',
                                                condition_parser = 'pandas')

In [ ]:
##### EXPECTATION 1 #####
# expectations for nothing to be null in any column of donations details
donations_mv.expect_column_values_to_not_be_null(column='donation_year')

In [ ]:
##### EXPECTATION 1 #####
# expectations for nothing to be null in any column of donations details
donations_mv.expect_column_values_to_not_be_null(column='recipient')

In [ ]:
##### EXPECTATION 1 #####
# expectations for nothing to be null in any column of donations details
donations_mv.expect_column_values_to_not_be_null(column='political_entity')

For these:

- excluded rows where donor name is "Contributions of 200/20 or less..." b/c these are aggregates
- excluded estate of and leadership because those are exceptions
- TO DO - ask about the leadership political entity filter - should this be "contestant|Contestant"

In [ ]:
##### EXPECTATION 2 #####
# note - no limit for Newfoundland, Yukon, or Saskatchewan so these are excluded from this expectation
# FEDERAL
donations_mv.expect_column_values_to_be_between(
    column = 'amount',
    max_value = 1675,
    row_condition = 'region=="Federal" & donor_full_name.str.contains("Contributions Of")==False & donor_full_name.str.contains("Estate Of")==False & donor_full_name.str.contains("Total Anonymous Contributions")==False & donation_year == 2022 & political_entity.str.contains("Leadership")==False',
    condition_parser = 'pandas'
)

In [ ]:
##### EXPECTATION 2 #####
# NEW BRUNSWICK
donations_mv.expect_column_values_to_be_between(
    column = 'amount',
    max_value = 3000,
    row_condition = 'region=="New Brunswick" & donor_full_name.str.contains("Contributions Of")==False & donor_full_name.str.contains("Estate Of")==False & donor_full_name.str.contains("Total Anonymous Contributions")==False & donation_year == 2022 & political_entity.str.contains("Leadership")==False',
    condition_parser = 'pandas'
)

In [ ]:
##### EXPECTATION 2 #####
# NOVA SCOTIA
donations_mv.expect_column_values_to_be_between(
    column = 'amount',
    max_value = 5000,
    row_condition = 'region=="Nova Scotia" & donor_full_name.str.contains("Contributions Of")==False & donor_full_name.str.contains("Estate Of")==False & donor_full_name.str.contains("Total Anonymous Contributions")==False & donation_year == 2022 & political_entity.str.contains("Leadership")==False',
    condition_parser = 'pandas'
)

In [ ]:
##### EXPECTATION 2 #####
# PEI
donations_mv.expect_column_values_to_be_between(
    column = 'amount',
    max_value = 3150,
    row_condition = 'region=="Prince Edward Island" & donor_full_name.str.contains("Contributions Of")==False & donor_full_name.str.contains("Estate Of")==False & donor_full_name.str.contains("Total Anonymous Contributions")==False & donation_year == 2022 & political_entity.str.contains("Leadership")==False',
    condition_parser = 'pandas'
)

In [ ]:
##### EXPECTATION 2 #####
# Ontario
donations_mv.expect_column_values_to_be_between(
    column = 'amount',
    max_value = 3325,
    row_condition = 'region=="Ontario" & donor_full_name.str.contains("Contributions Of")==False & donor_full_name.str.contains("Estate Of")==False & donor_full_name.str.contains("Total Anonymous Contributions")==False & donation_year == 2022 & political_entity.str.contains("Leadership")==False',
    condition_parser = 'pandas'
)

In [ ]:
##### EXPECTATION 2 #####
# Manitoba
donations_mv.expect_column_values_to_be_between(
    column = 'amount',
    max_value = 5000,
    row_condition = 'region=="Manitoba" & donor_full_name.str.contains("Contributions Of")==False & donor_full_name.str.contains("Estate Of")==False & donor_full_name.str.contains("Total Anonymous Contributions")==False & donation_year == 2022 & political_entity.str.contains("Leadership")==False',
    condition_parser = 'pandas'
)

In [ ]:
##### EXPECTATION 2 #####
# Alberta
donations_mv.expect_column_values_to_be_between(
    column = 'amount',
    max_value = 4300,
    row_condition = 'region=="Alberta" & donor_full_name.str.contains("Contributions Of")==False & donor_full_name.str.contains("Estate Of")==False & donor_full_name.str.contains("Total Anonymous Contributions")==False & donation_year == 2022 & political_entity.str.contains("Leadership")==False',
    condition_parser = 'pandas'
)

In [ ]:
##### EXPECTATION 2 #####
# BC
donations_mv.expect_column_values_to_be_between(
    column = 'amount',
    max_value = 1309.09,
    row_condition = 'region=="British Columbia" & donor_full_name.str.contains("Contributions Of")==False & donor_full_name.str.contains("Estate Of")==False & donor_full_name.str.contains("Total Anonymous Contributions")==False & donation_year == 2022 & political_entity.str.contains("Leadership")==False',
    condition_parser = 'pandas'
)

In [ ]:
##### EXPECTATION 2 #####
# Quebec
# this one is a bit tricky - depends on year, should I just make the max 500?
donations_mv.expect_column_values_to_be_between(
    column = 'amount',
    max_value = 500,
    row_condition = 'region=="Quebec" & donor_full_name.str.contains("Contributions Of")==False & donor_full_name.str.contains("Estate Of")==False & donor_full_name.str.contains("Total Anonymous Contributions")==False & donation_year == 2022 & political_entity.str.contains("Leadership")==False',
    condition_parser = 'pandas'
)

In [ ]:
##### EXPECTATION 2 #####
# Northwest Territories
donations_mv.expect_column_values_to_be_between(
    column = 'amount',
    max_value = 1500,
    row_condition = 'region=="Northwest Territories" & donor_full_name.str.contains("Contributions Of")==False & donor_full_name.str.contains("Estate Of")==False & donor_full_name.str.contains("Total Anonymous Contributions")==False & donation_year == 2022 & political_entity.str.contains("Leadership")==False',
    condition_parser = 'pandas'
)

In [ ]:
##### EXPECTATION 2 #####
# Nunavut
donations_mv.expect_column_values_to_be_between(
    column = 'amount',
    max_value = 2500,
    row_condition = 'region=="Nunavut"& donor_full_name.str.contains("Contributions Of")==False & donor_full_name.str.contains("Estate Of")==False & donor_full_name.str.contains("Total Anonymous Contributions")==False & donation_year == 2022 & political_entity.str.contains("Leadership")==False',
    condition_parser = 'pandas'
)

In [ ]:
### checking exceptions
donations_mv.query('region=="British Columbia" & donor_full_name.str.contains("Contributions Of")==False & donor_full_name.str.contains("Estate Of")==False & donor_full_name.str.contains("Total Anonymous Contributions")==False & donation_year == 2022 & political_entity.str.contains("Leadership")==False & amount > 1309.09')

donations_mv.query('region=="Quebec" & donor_full_name.str.contains("Contributions Of")==False & donor_full_name.str.contains("Estate Of")==False & donor_full_name.str.contains("Total Anonymous Contributions")==False & donation_year == 2022 & political_entity.str.contains("Leadership")==False & amount>500')

In [ ]:
##### EXPECTATION 3 #####
# run expectation
donations_mv.expect_column_values_to_be_between(
    column = 'diff_abs',
    max_value = 5,
    row_condition = 'donation_year>2000 & expect_total.isna()==False',
    condition_parser = 'pandas'
)

In [ ]:
donations_mv[(donations_mv.diff_abs > 1) & (donations_mv.expect_total > 0)].sort_values(['diff_abs'], ascending=False)

In [ ]:
donations_mv[(donations_mv.diff_abs > 0) & (donations_mv.expect_total.notna()) & (donations_mv.donation_year>2000)].shape[0]

In [ ]:
# exceptions for the year 2000 are b/c all monetary and non monerary values are both 0. so we exclude the year 2000's data from our test
donations_mv.query('region=="Federal" & donation_year>=2000 & expect_total.isna()==False & diff_abs>5')[['amount_monetary', 'amount_non_monetary']].drop_duplicates()

In [ ]:
# expectations across all databases
# date to match regex format
donations_mv.expect_column_values_to_match_regex(column = 'donation_date',
                                              regex = '\\d{4}-\\d{2}-\\d{2}',
                                              row_condition = "donation_date.isna()==False",
                                              condition_parser = 'pandas')

In [ ]:
# expectations across all databases
# added to never be null
donations_mv.expect_column_values_to_not_be_null(column='added')

In [ ]:
# expectations across all databases
# define set for comparison
regions = set(['British Columbia', 'Ontario', 'Northwest Territories', 
           'Alberta', 'Federal', 'Yukon', 'Saskatchewan', 'Manitoba',
           'Quebec', 'Newfoundland and Labrador', 'Nova Scotia',
           'New Brunswick', 'Nunavut', 'Prince Edward Island'])

# expect all region values to be within regions set
donations_mv.expect_column_values_to_be_in_set(column = 'region',
                                               value_set = regions)

In [ ]:
# grab expectation suite with all expectations (not just those with 100% success)
donations_mv.get_expectation_suite(discard_failed_expectations=False)

In [ ]:
import json

with open( "donations_expectations.json", "w") as my_file:
    my_file.write(
        json.dumps(donations_mv.get_expectation_suite(discard_failed_expectations=False).to_json_dict())
    )

In [ ]:
# expect donor full name to not be all capital letters
# unsure about regex pattern here - but the expectation was a success so that means have no two consecutive uppercase letters in a name
donations_mv.expect_column_values_to_not_match_regex(column='donor_full_name',
                                                 regex = "[A-Z]{2,}")

In [ ]:
# expect donor full name to be in format first name last name (not last name comma first name)
donations_mv.expect_column_values_to_not_match_regex(column='donor_full_name',
                                                 regex = ", ")